In [1]:
#import os
import warnings

warnings.filterwarnings("ignore")

#os.chdir("../../..")

In [2]:
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
import torch
import numpy as np
from pytorch_forecasting import Baseline, NBeats, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.data.examples import generate_ar_data
from pytorch_forecasting.metrics import SMAPE

In [3]:
df1 = pd.read_csv('../all_series.csv')
df1.head()

FileNotFoundError: [Errno 2] No such file or directory: '../all_series.csv'

In [6]:
from pytorch_forecasting import TimeSeriesDataSet

max_encoder_length=1095  # This is the lookbackwindow
max_prediction_length=365 # This is the prediction window / horizon 

training_cutoff = df1["time_idx"].max() - max_prediction_length


# create the dataset from the pandas dataframe
training = TimeSeriesDataSet(
    df1,
    group_ids=["group"],
    target="value",
    time_idx="time_idx",
    #min_encoder_length=200,
    max_encoder_length=max_encoder_length,
    #min_prediction_length=50,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=["value"],
)

validation = TimeSeriesDataSet.from_dataset(training, df1, min_prediction_idx=training_cutoff + 1)
batch_size = 1024 # Aus dem Paper 
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

In [7]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
print(SMAPE()(baseline_predictions, actuals))
(actuals - baseline_predictions).abs().mean()

tensor(0.3471)


C:\Users\strub\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:261: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
C:\Users\strub\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:261: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(
C:\Users\strub\anaconda3\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (SMAPE). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can

tensor(0.3223)

In [22]:
pl.seed_everything(42)
trainer = pl.Trainer(gpus=0, gradient_clip_val=0.01)
net = NBeats.from_dataset(training, learning_rate=3e-2, weight_decay=1e-2, widths=[32, 512], backcast_loss_ratio=0.1)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### Find best learnin rate

In [148]:
#res = trainer.tuner.lr_find(net, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader, min_lr=1e-5)
#print(f"suggested learning rate: {res.suggestion()}")
#fig = res.plot(show=True, suggest=True)
#fig.show()
#net.hparams.learning_rate = res.suggestion()

### Fit Model

In [23]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")

trainer = pl.Trainer(
    max_epochs=5000, #iterations in N-Beats Paper = 5.000 
    gpus=0,
    enable_model_summary=True,
    gradient_clip_val=0.01,
    callbacks=[early_stop_callback],# Model hört auf zu lernen, wenn es sich nicht weiter verbessert
    limit_train_batches=30,
)


net = NBeats.from_dataset(
    training,
    learning_rate=0.001, # Aud dem Paper
    log_interval=10,
    log_val_interval=1,
    weight_decay=1e-2,
    widths=[256, 2048], # aus N-Beats Paper 
    backcast_loss_ratio=1.0,
)

trainer.fit(
    net,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name            | Type       | Params
-----------------------------------------------
0 | loss            | MASE       | 0     
1 | logging_metrics | ModuleList | 0     
2 | net_blocks      | ModuleList | 40.5 M
-----------------------------------------------
40.5 M    Trainable params
0         Non-trainable params
40.5 M    Total params
161.811   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

### Evaluate

In [8]:
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = NBeats.load_from_checkpoint(best_model_path)

NameError: name 'trainer' is not defined

In [25]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_model.predict(val_dataloader)
print(SMAPE()(predictions, actuals))
(actuals - predictions).abs().mean()

tensor(0.1060)


tensor(0.0899)

In [26]:
raw_predictions, x = best_model.predict(val_dataloader, mode="raw", return_x=True)

In [3]:
for idx in range(7):  # plot 10 examples
    best_model.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True);

NameError: name 'best_model' is not defined

In [2]:
for idx in range(7):  # plot 10 examples
    best_model.plot_interpretation(x, raw_predictions, idx=idx);

NameError: name 'best_model' is not defined

In [1]:
x_value=raw_predictions.seasonality
x=x_value.cpu().detach().numpy()
seasonality=np.reshape(x,(1095,))
seasonality
print(len(seasonality))
seasonality


NameError: name 'raw_predictions' is not defined

In [125]:
series = pd.date_range(start='2016-01-01', end='2021-12-31', freq='D')
df_prediction = pd.DataFrame(columns=['date', 'trend', 'yearly', 'weekly'])
df_prediction['yearly'] = seasonality
df_prediction.head()

,date,trend,yearly,weekly
0,NaN,NaN,0.559919,NaN
1,NaN,NaN,0.565740,NaN
2,NaN,NaN,0.566699,NaN
3,NaN,NaN,0.568277,NaN
4,NaN,NaN,0.573263,NaN


In [188]:
print(x)
print(np.shape(raw_predictions.backcast))
np.shape(raw_predictions)

{'encoder_cat': tensor([], size=(1, 730, 0), dtype=torch.int64), 'encoder_cont': tensor([[[-0.0099],
         [-0.0640],
         [ 0.3645],
         [ 0.2245],
         [ 0.1229],
         [ 0.6278],
         [ 0.1605],
         [ 0.2700],
         [ 0.0054],
         [ 0.4645],
         [ 0.2078],
         [ 0.2811],
         [ 0.6010],
         [ 0.5001],
         [-0.0674],
         [ 0.7069],
         [ 0.1801],
         [ 0.1591],
         [ 0.5899],
         [ 0.6820],
         [ 0.2926],
         [ 0.6585],
         [ 0.1855],
         [ 0.2150],
         [ 0.7394],
         [ 0.2738],
         [ 1.0900],
         [ 0.4443],
         [ 0.8170],
         [ 1.0465],
         [ 0.6112],
         [ 0.7755],
         [ 0.8977],
         [ 0.4382],
         [ 0.4644],
         [ 0.6863],
         [ 1.2351],
         [ 1.0291],
         [ 0.5534],
         [ 0.4529],
         [ 0.9525],
         [ 1.3093],
         [ 0.7102],
         [ 0.6870],
         [ 0.6632],
         [ 1.2576],

(5,)

In [189]:
df1[1820:1827]

,group,time_idx,value
1820,0,1820,0.401751
1821,0,1821,0.409281
1822,0,1822,0.578859
1823,0,1823,0.539713
1824,0,1824,0.531257
1825,0,1825,0.618593
1826,0,1826,0.535315
